In [16]:
import numpy as np
from abc import ABC, abstractmethod

In [17]:
class Predictor(ABC):
    def __init__(self, model_filename):
        self.model_filename = model_filename
        self.model = None
        self.model = self.loadModel()

    @abstractmethod
    def loadModel(self):
        pass

    @abstractmethod
    def predict(self, inputs, n_to_predict):
        pass

In [18]:
class XGBoostPredictor(Predictor):
    def loadModel(self):
        
        import xgboost as xgb

        model = xgb.Booster()
        model.load_model(self.model_filename)
        return model
    
    def predict(self, inputs, n_to_predict):

        import xgboost as xgb
        
        from sklearn.preprocessing import MinMaxScaler
        scaler=MinMaxScaler(feature_range=(0,1))
        
        new_dataset = inputs.copy()
        new_dataset.set_index("timestamp", inplace=True)
        
        final_dataset=new_dataset.values
        scaler=MinMaxScaler(feature_range=(0,1))    
        scaler.fit_transform(final_dataset)

        inputs_data = final_dataset[-160:]
        inputs_data=inputs_data.reshape(-1,1)
        inputs_data=scaler.transform(inputs_data)
        
        X_test=[]
        for i in range(60,inputs_data.shape[0]):
            X_test.append(inputs_data[i-60:i,0])
        X_test = xgb.DMatrix(X_test)
        
        closing_price=self.model.predict(X_test)
        closing_price=closing_price.reshape(-1, 1)
        closing_price=scaler.inverse_transform(closing_price)
        
        return closing_price

In [19]:
from tensorflow.keras.models import load_model

class RNNPredictor(Predictor):
    def loadModel(self):
        return load_model(self.model_filename)
    
    def predict(self, inputs, n_to_predict):
        
        from sklearn.preprocessing import MinMaxScaler
        scaler=MinMaxScaler(feature_range=(0,1))
        
        new_dataset = inputs.copy()
        new_dataset.set_index("timestamp", inplace=True)
        
        final_dataset=new_dataset.values
        scaler=MinMaxScaler(feature_range=(0,1))    
        scaler.fit_transform(final_dataset)

        inputs_data = final_dataset[-160:]
        inputs_data=inputs_data.reshape(-1,1)
        inputs_data=scaler.transform(inputs_data)
        
        X_test=[]
        for i in range(60,inputs_data.shape[0]):
            X_test.append(inputs_data[i-60:i,0])
        X_test=np.array(X_test)

        X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
        closing_price=self.model.predict(X_test)
        closing_price=scaler.inverse_transform(closing_price)
        
        return closing_price

In [20]:
class LSTMPredictor(Predictor):
    def loadModel(self):    
        return load_model(self.model_filename)
    
    def predict(self, inputs, n_to_predict):
        
        from sklearn.preprocessing import MinMaxScaler
        scaler=MinMaxScaler(feature_range=(0,1))
        
        new_dataset = inputs.copy()
        new_dataset.set_index("timestamp", inplace=True)
        
        final_dataset=new_dataset.values
        scaler=MinMaxScaler(feature_range=(0,1))
        scaler.fit_transform(final_dataset)

        inputs_data = final_dataset[-160:]
        inputs_data=inputs_data.reshape(-1,1)
        inputs_data=scaler.transform(inputs_data)
        
        X_test=[]
        for i in range(60,inputs_data.shape[0]):
            X_test.append(inputs_data[i-60:i,0])
        X_test=np.array(X_test)
        X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
        
        closing_price=self.model.predict(X_test)
        closing_price=scaler.inverse_transform(closing_price)
        
        return closing_price